# Multiword expressions identification and extraction

In [80]:
import spacy
import pandas as pd
import re
import os

from random import sample
from collections import Counter
import numpy as np  


In [69]:
path_to_files = "ustawy"

def read_normalized_documents(): 
    file_names = os.listdir(path_to_files)
    return {
        name: _normalize_document(_read_document(name, path_to_files))
        for name in file_names
        if name.endswith(".txt")
    }

def _read_document(name: str, path: str):
    with open(os.path.join(path, name), 'r') as f:
        return f.read()


def _normalize_document(document: str):
    return re.sub(r"\s+", " ", document).lower()

In [5]:
nlp = spacy.load("pl_core_news_sm")  # had to be downloaded separately with "python3 -m spacy download pl_core_news_sm"
tokenizer = nlp.tokenizer
documents = read_normalized_documents()

In [70]:
tokenized_documents = {name: [t.text for t in tokenizer(text)] for name, text in documents.items()}
print(type(tokenized_documents))
for name, tokens in sample(list(tokenized_documents.items()), 10):
    print(f"{name}: {sample(tokens, 10)}", type(tokens))

<class 'dict'>
2001_994.txt: ['994', 'przez', 'art', 'dnia', 'przeciwko', 'wchodzi', 'zgromadzenie', 'przyjętej', 'poz', 'konwencji'] <class 'list'>
1999_700.txt: [',', '.', 'życie', 'nr', '1999', '1999', 'ogłoszenia', ',', 'poz', 'grudnia'] <class 'list'>
1998_717.txt: ['33', 'rozstrzygnięcie', ',', 'o', 'samorządową', 'przysługuje', 'sprawie', '§', 'i', 'czynnej'] <class 'list'>
1996_776.txt: ['dostępna', 'w', 'wprowadzenia', 'o', '.', 'skargi', '2', 'dokonać', 'wynosi', 'aby'] <class 'list'>
2004_1055.txt: ['19a', 'pierwszego', ';', 'niezbędnym', 'urbaniście', ',', 'zawodowych', 'krajowa', 'polskiej', 'życie'] <class 'list'>
1999_527.txt: ['.', 'od', 'siedzibę', '.', 'przewodniczący', '1998', 'nr', 'w', 'nr', 'w'] <class 'list'>
2004_2783.txt: ['i', 'r', ',', 'nawzajem', 'budynku', ':', 'przyjęcia', '.', 'tych', 'użytkowej'] <class 'list'>
2001_1197.txt: ['gmin', 'tonę', 'zaspokojenia', 'prawnych', 'zarządzającego', 'skład', 'użytkowanie', ')', 'art', 'portem'] <class 'list'>
2003_8

## 2. Compute bigram counts of downcased tokens

In [71]:
bigrams = [
    (f, s)
    for tokens in tokenized_documents.values()
    for (f, s) in zip(tokens[:-1], tokens[1:])
]
print(type(bigrams),bigrams[0:100])

<class 'list'> [(' ', 'dz.u'), ('dz.u', '.'), ('.', 'z'), ('z', '2001'), ('2001', 'r'), ('r', '.'), ('.', 'nr'), ('nr', '81'), ('81', ','), (',', 'poz'), ('poz', '.'), ('.', '874'), ('874', 'ustawa'), ('ustawa', 'z'), ('z', 'dnia'), ('dnia', '21'), ('21', 'czerwca'), ('czerwca', '2001'), ('2001', 'r'), ('r', '.'), ('.', 'o'), ('o', 'zmianie'), ('zmianie', 'ustawy'), ('ustawy', 'o'), ('o', 'państwowej'), ('państwowej', 'straży'), ('straży', 'pożarnej'), ('pożarnej', 'art'), ('art', '.'), ('.', '1'), ('1', '.'), ('.', 'w'), ('w', 'ustawie'), ('ustawie', 'z'), ('z', 'dnia'), ('dnia', '24'), ('24', 'sierpnia'), ('sierpnia', '1991'), ('1991', 'r'), ('r', '.'), ('.', 'o'), ('o', 'państwowej'), ('państwowej', 'straży'), ('straży', 'pożarnej'), ('pożarnej', '('), ('(', 'dz.u'), ('dz.u', '.'), ('.', 'nr'), ('nr', '88'), ('88', ','), (',', 'poz'), ('poz', '.'), ('.', '400'), ('400', 'z'), ('z', '1992'), ('1992', 'r'), ('r', '.'), ('.', 'nr'), ('nr', '21'), ('21', ','), (',', 'poz'), ('poz', '.')

In [72]:
# Calculate bigrams count

bigrams_counts = Counter(bigrams)

bigrams_counts.most_common(10)

[(('art', '.'), 83778),
 (('ust', '.'), 53552),
 (('poz', '.'), 45198),
 ((',', 'poz'), 43188),
 (('.', '1'), 39927),
 (('-', '-'), 36547),
 (('r', '.'), 33010),
 (('w', 'art'), 32042),
 ((',', 'o'), 29926),
 (('mowa', 'w'), 28471)]

## 3. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.

In [74]:
def isalpha(s):
    for letter in s:
        if not letter.isalpha():  # is this characted alphanumeric?
            return False
    return True 


bigrams_alpha_total = Counter({x: count for x, count in bigrams_counts.items() if isalpha(x[0]) and isalpha(x[1])})

bigrams_alpha_total.most_common(10)

[(('w', 'art'), 32042),
 (('mowa', 'w'), 28471),
 (('w', 'ust'), 23557),
 (('o', 'których'), 13884),
 (('których', 'mowa'), 13857),
 (('otrzymuje', 'brzmienie'), 9553),
 (('z', 'dnia'), 9527),
 (('o', 'którym'), 9184),
 (('którym', 'mowa'), 9171),
 (('do', 'spraw'), 8715)]

## 4. Use pointwise mutual information to compute the measure for all pairs of words.

PMI for 2 elements:
```
log2(p(a,b) / ( p(a) * p(b) )) =  log2(p(a,b)) - log2( p(a) + p(b) )
```

In [75]:
# Calculate total number of words for word's probability, for this I have to calculate unigrams:

unigrams = [
    f
    for tokens in tokenized_documents.values()
    for f in tokens
]
print(type(unigrams),unigrams[0:100])

unigrams_alpha_total = Counter({x: count for x, count in Counter(unigrams).items() if isalpha(x)})
unigrams_alpha_total.most_common(10)


<class 'list'> [' ', 'dz.u', '.', 'z', '2001', 'r', '.', 'nr', '81', ',', 'poz', '.', '874', 'ustawa', 'z', 'dnia', '21', 'czerwca', '2001', 'r', '.', 'o', 'zmianie', 'ustawy', 'o', 'państwowej', 'straży', 'pożarnej', 'art', '.', '1', '.', 'w', 'ustawie', 'z', 'dnia', '24', 'sierpnia', '1991', 'r', '.', 'o', 'państwowej', 'straży', 'pożarnej', '(', 'dz.u', '.', 'nr', '88', ',', 'poz', '.', '400', 'z', '1992', 'r', '.', 'nr', '21', ',', 'poz', '.', '86', 'i', 'nr', '54', ',', 'poz', '.', '254', ',', 'z', '1994', 'r', '.', 'nr', '53', ',', 'poz', '.', '214', ',', 'z', '1995', 'r', '.', 'nr', '4', ',', 'poz', '.', '17', 'i', 'nr', '34', ',', 'poz', '.', '163']


[('w', 201200),
 ('i', 90006),
 ('art', 83804),
 ('z', 82438),
 ('o', 64776),
 ('do', 60732),
 ('ust', 53636),
 ('na', 50643),
 ('się', 45886),
 ('lub', 45800)]

In [76]:
total = sum(unigrams_alpha_total.values())
print("Total:", total)

Total: 3566806


In [87]:
def p(word_occ, total):
    return word_occ / total

def pmi2(word1, word2, total):
    word1_occ = unigrams_alpha_total[word1]
    word2_occ = unigrams_alpha_total[word2]
    together_occ = bigrams_alpha_total[(word1, word2)]
    return np.log2(p(together_occ, total)) - np.log2( p(word1_occ, total)) - np.log2(p(word2_occ, total))

In [88]:
bigrams_total_pmi = {bigram: pmi2(bigram[0], bigram[1], total=total) for bigram, _  in bigrams_alpha_total.items()}


## 5. Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [89]:
Counter(bigrams_total_pmi).most_common()[:10]

[(('doktorów', 'habilitowanych'), 21.76620131850449),
 (('pionową', 'ścianę'), 21.76620131850449),
 (('inte', 'gracyjnymi'), 21.76620131850449),
 (('techno', 'logii'), 21.76620131850449),
 (('usprawnianie', 'zaburzonych'), 21.76620131850449),
 (('przepro', 'wadza'), 21.76620131850449),
 (('gałki', 'ocznej'), 21.76620131850449),
 (('stępkę', 'położono'), 21.76620131850449),
 (('wybuchła', 'wojna'), 21.76620131850449),
 (('dało', 'pożytecznego'), 21.76620131850449)]

## 6. Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences).

In [91]:
bigrams_filtered_pmi = {bigram: pmi2(bigram[0], bigram[1], total=total) for bigram, count in bigrams_alpha_total.items() if count >= 5}
bigrams_pmi_top10 = Counter(bigrams_filtered_pmi).most_common()[:10]
bigrams_pmi_top10

[(('ręcznego', 'miotacza'), 19.444273223617127),
 (('stajnią', 'wyścigową'), 19.444273223617127),
 (('świeckie', 'przygotowujące'), 19.444273223617127),
 (('klęskami', 'żywiołowymi'), 19.444273223617127),
 (('obcowania', 'płciowego'), 19.444273223617127),
 (('nietykalność', 'cielesną'), 19.444273223617127),
 (('grzegorz', 'schetyna'), 19.444273223617127),
 (('młynki', 'młotkowe'), 19.444273223617127),
 (('młyny', 'kulowe'), 19.444273223617127),
 (('najnowszych', 'zdobyczy'), 19.444273223617127)]

## 7. Use KRNNT or Clarin-PL API(https://ws.clarin-pl.eu/tager.shtml) to tag and lemmatize the corpus.

Used Clarin-PL API with Morfeusz2, bulk upload of 'ustawy.zip'

In [21]:
import xml.etree.ElementTree as ET
import tqdm

from pathlib import Path 

In [23]:
tag_tokenized_acts = {}

path_to_results = Path("Wynik")
n_acts = len(list(path_to_results.iterdir()))

for act in tqdm.tqdm(path_to_results.iterdir(), desc="Tokenizing tagged acts", total=n_acts):
    act_id = act.stem
    content = act.read_text(encoding="utf8")
    tree = ET.fromstring(text=content)
    tag_tokens = []
    for tok in tree.iter("tok"):
        lex = tok.find("lex")
        base = lex.find("base").text
        ctag = lex.find("ctag").text
        tag_token = f"{base.lower()}:{ctag.split(':')[0]}"
        tag_tokens.append(tag_token)
    tag_tokenized_acts[act_id] = tag_tokens

Tokenizing tagged acts: 100%|██████████| 1179/1179 [03:48<00:00,  5.17it/s]


## 8. Using the tagged corpus compute bigram statistic for the tokens containing: a. lemmatized, downcased word b. morphosyntactic category of the word (subst, fin, adj, etc.)

In [24]:
clarin_bigrams = [
    (f, s)
    for tag_tokens in tag_tokenized_acts.values()
    for (f, s) in zip(tag_tokens[:-1], tag_tokens[1:])
]

Filter for real words

In [26]:
def is_word(token: str):
    parts = token.split(":")
    if len(parts) != 2:
        return False
    else:
        return isalpha(parts[0])


In [27]:
tag_bigrams_counter = Counter(clarin_bigrams)
tag_bigrams_counter.most_common(10)

[(('art:ign', '.:interp'), 83779),
 (('usta:subst', '.:interp'), 53553),
 (('poz:ign', '.:interp'), 45222),
 ((',:interp', 'poz:ign'), 43191),
 (('.:interp', '1:num'), 39987),
 (('-:interp', '-:interp'), 36548),
 (('r:ign', '.:interp'), 33029),
 (('w:prep', 'art:ign'), 32044),
 ((',:interp', 'o:prep'), 29926),
 (('o:prep', 'który:adj'), 28656)]

In [28]:
clarin_bigrams_alpha = (
    (f, s)
    for tag_tokens in tag_tokenized_acts.values()
    for (f, s) in zip(tag_tokens[:-1], tag_tokens[1:])
    if is_word(f) and is_word(s)
)

In [29]:
clarin_bigrams_alpha_total = Counter(clarin_bigrams_alpha)
clarin_bigrams_alpha_total.most_common(10)

[(('w:prep', 'art:ign'), 32044),
 (('o:prep', 'który:adj'), 28656),
 (('który:adj', 'mowa:subst'), 28538),
 (('mowa:subst', 'w:prep'), 28473),
 (('w:prep', 'usta:subst'), 23557),
 (('z:prep', 'dzień:subst'), 11360),
 (('otrzymywać:fin', 'brzmienie:subst'), 10536),
 (('określony:adj', 'w:prep'), 10240),
 (('do:prep', 'sprawa:subst'), 8718),
 (('ustawa:subst', 'z:prep'), 8625)]

In [37]:
clarin_unigrams_alpha_total = Counter(
    tag_token for tag_tokens in tag_tokenized_acts.values() for tag_token in tag_tokens if is_word(tag_token))

clarin_unigrams_alpha_total.most_common(10)


[('w:prep', 202951),
 ('i:conj', 90044),
 ('z:prep', 87991),
 ('art:ign', 83805),
 ('o:prep', 64809),
 ('do:prep', 60768),
 ('usta:subst', 53641),
 ('na:prep', 50657),
 ('który:adj', 49382),
 ('się:qub', 45887)]

Calculating PMIs:

In [96]:
def clarin_pmi2(word1, word2, total):
    word1_occ = clarin_unigrams_alpha_total[word1]
    word2_occ = clarin_unigrams_alpha_total[word2]
    together_occ = clarin_bigrams_alpha_total[(word1, word2)]
    return np.log2(p(together_occ, total)) - np.log2(p(word1_occ, total)) - np.log2(p(word2_occ, total))

In [97]:
clarin_total = sum(clarin_unigrams_alpha_total.values())
print("Clarin total:", clarin_total)
clarin_bigrams_total_pmi = {bigram: clarin_pmi2(bigram[0], bigram[1], total=clarin_total) \
    for bigram, _  in clarin_bigrams_alpha_total.items()}

Clarin total: 3578037


In [98]:
Counter(clarin_bigrams_total_pmi).most_common()[:10]

[(('atrakcyjny:adj', 'turystycznie:adv'), 21.77073687550676),
 (('niepeł:ign', 'nosprawności:ign'), 21.77073687550676),
 (('dzonej:ign', 'obdukcja:subst'), 21.77073687550676),
 (('inwestycy:ign', 'jnych:ign'), 21.77073687550676),
 (('okr:ign', 'eśl:ign'), 21.77073687550676),
 (('pos:ign', 'taca:subst'), 21.77073687550676),
 (('zwi:ign', 'ązk:ign'), 21.77073687550676),
 (('ier:ign', 'ają:ign'), 21.77073687550676),
 (('ają:ign', 'cyc:subst'), 21.77073687550676),
 (('potasowyc:ign', 'atmosfe:ign'), 21.77073687550676)]

## 10. Compute the same statistics as for the non-lemmatized words (i.e. PMI) and print top-10 entries with at least 5 occurrences.

In [99]:
clarin_bigrams_filtered_pmi = {bigram: clarin_pmi2(bigram[0], bigram[1], total=clarin_total) \
    for bigram, count in clarin_bigrams_alpha_total.items() if count >= 5}
clarin_bigrams_pmi_top10 = Counter(clarin_bigrams_filtered_pmi).most_common()[:10]
clarin_bigrams_pmi_top10


[(('młynek:subst', 'młotkowy:adj'), 19.448808780619398),
 (('grzegorz:subst', 'schetyna:ign'), 19.448808780619398),
 (('teryto:ign', 'rialnego:ign'), 19.448808780619398),
 (('pasta:subst', 'emulsyjny:adj'), 19.185774374785606),
 (('chrom:subst', 'sześciowartościowy:adj'), 19.185774374785606),
 (('odpowiedzieć:fin', 'dzialności:ign'), 19.185774374785606),
 (('adam:subst', 'mickiewicz:subst'), 19.185774374785606),
 (('łańcuchowa:subst', 'rozszczepienie:subst'), 19.185774374785606),
 (('młyn:subst', 'kulowy:adj'), 18.963381953449158),
 (('piotrek:subst', 'trybunalski:adj'), 18.963381953449158)]

## 11. Compute trigram counts for both corpora and perform the same filtering.

### A. The corpus tokenized by SpaCy:

In [65]:
trigrams_alpha = [
    (f, s, t)
    for tokens in tokenized_documents.values()
    for (f, s, t) in zip(tokens[:-2], tokens[1:-1],tokens[2:])
    if isalpha(f) and isalpha(s) and isalpha(t)
]

trigrams_alpha_total = Counter(trigrams_alpha)
trigrams_alpha_total.most_common(10)

[(('o', 'których', 'mowa'), 13856),
 (('których', 'mowa', 'w'), 13806),
 (('mowa', 'w', 'ust'), 13474),
 (('mowa', 'w', 'art'), 12311),
 (('o', 'którym', 'mowa'), 9169),
 (('którym', 'mowa', 'w'), 9147),
 (('o', 'której', 'mowa'), 5510),
 (('której', 'mowa', 'w'), 5487),
 (('w', 'drodze', 'rozporządzenia'), 4685),
 (('właściwy', 'do', 'spraw'), 4620)]

### B. The corpus lemmatized by CLARIN

In [100]:
clarin_trigrams_alpha = [
    (f, s, t)
    for tokens in tag_tokenized_acts.values()
    for (f, s, t) in zip(tokens[:-2], tokens[1:-1],tokens[2:])
    if is_word(f) and is_word(s) and is_word(t)
]

clarin_trigrams_alpha_total = Counter(clarin_trigrams_alpha)
clarin_trigrams_alpha_total.most_common(10)

[(('o:prep', 'który:adj', 'mowa:subst'), 28535),
 (('który:adj', 'mowa:subst', 'w:prep'), 28442),
 (('mowa:subst', 'w:prep', 'usta:subst'), 13474),
 (('mowa:subst', 'w:prep', 'art:ign'), 12311),
 (('ustawa:subst', 'z:prep', 'dzień:subst'), 8589),
 (('właściwy:adj', 'do:prep', 'sprawa:subst'), 7966),
 (('minister:subst', 'właściwy:adj', 'do:prep'), 7888),
 (('w:prep', 'droga:subst', 'rozporządzenie:subst'), 4751),
 (('zastępować:fin', 'się:qub', 'wyraz:subst'), 3653),
 (('w:prep', 'ustawa:subst', 'z:prep'), 3646)]

## 12. Use PMI (with 5 occurrence threshold) to compute top 10 results for the trigrams. Devise a method for computing the values, based on the results for bigrams.

PMI for 3 elements:
```
log2( p(a,b,c) / ( p(a) * p(b) * p(c) ) ) = log2(p(a,b,c)) - log2(p(a)) - log2(p(b)) - log2(p(c))
```

In [ ]:
def pmi3(word1, word2, word3, total):
    word1_occ = unigrams_alpha_total[word1]
    word2_occ = unigrams_alpha_total[word2]
    word3_occ = unigrams_alpha_total[word3]
    together_occ = trigrams_alpha_total[(word1, word2, word3)]
    return np.log2(p(together_occ, total)) - np.log2( p(word1_occ, total)) - np.log2(p(word2_occ, total)) - np.log2(p(word3_occ, total)) 
